<center><h1> Security Library </h1></center><br><br>

<h3> I. INTRODUCTION </h3>
    
   In this notebook, we're going to build a small security library compatible with the resources of IoT, while integrating the ins and outs of security in this field. When we develop techniques for protecting and securing digital data, we have to make sure the adversary has no way of breaching it. Thus, probability and complexity play an important role in cryptography.
    
   This is why we must first be able to generate random numbers and prime numbers :
1. Prime numbers will be the secret keys,
    - whose product is large enough to make it impossible, in practice, to find its primary factors (the keys)
    - whose security is guaranteed by this complexity of finding the prime factors of a number,
2. Random numbers will be of two types:
    - the true random numbers, produced materially by the sensors, not reproducible, will be used to generate these encryption keys.
    - pseudo-random numbers, produced by algorithms, which are reproducible, will be used to mask messages, in a symmetrical encryption method.

<h3>II. A SYMMETRICAL ENCRYPTION ALGORITHM FOR IOT </h3>


<h6>THE ONE-TIME PAD</h6>

Let's start first by illustrating these remarks, by presenting the symmetrical encryption known as one-time pad. In cryptography, the one-time pad (OTP) is an encryption technique that cannot be cracked, but requires the use of a single-use pre-shared key that is no smaller than the message being sent. In this technique, a plaintext is paired with a random secret key (also referred to as a one-time pad). Then, each bit or character of the plaintext is encrypted by combining it with the corresponding bit or character from the pad using modular addition.

In [9]:
from random import SystemRandom
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

cryptogen = SystemRandom()

def symmetrical_encrypt(msg):
    mask = [cryptogen.randrange(2) for i in range(len(msg))]
    print(mask)
    encr = ""
    for i in range(len(msg)):
        res = (ord(msg[i]) - ord('0')) ^ mask[i]
        encr = encr + str(res)
    return ((encr, mask))

def symmetrical_decrypt(encr, mask):
     decr = ""
     for i in range(len(mask)):
        res = (ord(encr[i]) - ord('0')) ^ mask[i]
        decr = decr + str(res)
     return ((decr, mask))

<br><h3>III. CONSTITUTION OF RANDOM GENERATORS</h3>

Random and pseudo-random number generators are one of the foundations of computer security. The first ones are based on hardware, the second are based on algorithms.


<h6>1. LINEAR CONGRUENTIAL GENERATORS</h6>

A linear congruential generator (LCG) is an algorithm that yields a sequence of pseudo-randomized numbers calculated with a discontinuous piecewise linear equation. The method represents one of the oldest and best-known pseudorandom number generator algorithms.

Their very low complexity makes them usable in the Internet of Things, even at very low resources;
They are however unsafe, so other generators are later introduced. These LCGs are defined as follows:
Xn+1=(a⋅Xn+c)%m


In [10]:
def generate_next(a, c, m, Xn):
    res = (a*Xn + c) % m
    return (a*Xn + c) % m

def generate_seq(length):
    seq = []
    # find optimal a, c, m, seed
    (a, c, m, seed) = find_opt_params()
    seq.append(generate_next(a, c, m, seed))
    for i in range(1, length):
        seq.append(generate_next(a, c, m, seq[i-1]))
    return seq

def find_opt_params():
    m = pow(16, cryptogen.randrange(10))
    c = pow(3, cryptogen.randrange(10))
    a = 5
    seed = pow(11, cryptogen.randrange(10))
    return (a, c, m, seed)

<h6>2. THE XORSHIFT</h6>

Xorshift random number generators, also called shift-register generators are a class of pseudorandom number generators that were discovered by George Marsaglia.
1. We start from a 32-bit vector, which serves as a seed.
2. To calculate the new term from the current term x.
3. The value obtained after these three operations is the new term of the sequence, to be output.

In [11]:
def xorshift(a,b,c):

    x = 123456789
    y = 362436069
    z = 521288629
    w = 88675123

    def _random():
        nonlocal x, y, z, w
        t = x ^ ((x << b) & 0xFFFFFFFF)  # 32-bit
        x, y, z = y, z, w
        w = (w ^ (w >> a)) ^ (t ^ (t >> c))
        return w

    return _random

<br><h3>IV. EFFICIENT CALCULATION OF A POWER</h3>

Regular multiplication is O(n) with a very low constant factor with %m.
Whereas, this function calculates in O(log n) time in python but it takes a lot of time when numbers are large enough if you first calculate the value of xy and then mod it with p to get (xy) % p evaluated.

Fast exponentiation is a classical technique, used in practice, to obtain the power of a given number.
This fast exponentiation iterates, for the calculation of Xe, the following operations:

- the square elevation,
- multiplication by X,
in an order depending on the base 2 entry of e.

It is well understood in its recursive version :

- if e is even, we calculate (recursively) Xe/2, which we multiply by itself,
- otherwise, we calculate (recursively) Xe−1, which we multiply by X.

In [13]:
def fast_exp(message,X):
    word = getWord(message)
    print(word)
    
    #We start from the number X.
    res = X;
    for elem in word:
        #If the first letter is S, we raise to the square
        if elem == "S":
            res = res*res
        #Otherwise, multiply by X
        else:
            res = res*X
            
    return res

In [15]:
# Recursive version

def binpow (a,n):
    if (n == 0):
        return 1
    if (n % 2 == 1):
        return binpow (a, n-1) * a;
    else:
        b = binpow (a, n/2);
        return b * b;

<br><h3>V. GENERATION OF PRIME NUMBERS</h3>

Most current security approaches requires finding the prime factors of a large integer. Therefore, we must be able to find two large original prime numbers and make their product. This method is secure as it will be difficult to find the prime factors of this product.

<h6>1. ERATOSTHENES' SIEVE</h6>

To obtain a list of prime numbers below N, one can proceed as follows:

- Create the list of integers from 2 to N,
- Delete, in this list, the multiples of 2, then the multiples of the next remaining number in the list (3), then the multiples of the next remaining number in the list (5)...
- Stop at N/2.

In [27]:
def factor(n):
  if n==1:
      return set([])
  else:
      for k in range(2,n+1):
          if n%k == 0:
              L = factor(n/k)
  return L.union([k])

<h6>2. A FERMAT THEOREM</h6>

    If n is prime, then n divides an−a,a>1.  
    or an≡a[n].
To obtain a large prime number, the idea is to draw a large number of digits, concatenate them as a number, and then test if this number is prime.

In [26]:
def fermat_test(n, k):

    if n == 2:
        return True
    if n % 2 == 0:
        return False
    for i in xrange(k):
        a = random.randint(1, n-1)
        if pow(a, n-1) % n != 1:
            return False
    return True

<h6>3. MILLER-RABIN</h6>

Miller-Rabin is, in a way, a refinement of the above. It is once again a probabilistic test of primacy, but for which the risk of being wrong never happens in practice. This test is very effective in measuring the primality of large integers.

In [28]:
import random
 
# Utility function to do
# modular exponentiation.
# It returns (x^y) % p
def power(x, y, p):
     
    # Initialize result
    res = 1;
     
    # Update x if it is more than or
    # equal to p
    x = x % p;
    while (y > 0):
         
        # If y is odd, multiply
        # x with result
        if (y & 1):
            res = (res * x) % p;
 
        # y must be even now
        y = y>>1; # y = y/2
        x = (x * x) % p;
     
    return res;
 
# This function is called
# for all k trials. It returns
# false if n is composite and
# returns false if n is
# probably prime. d is an odd
# number such that d*2<sup>r</sup> = n-1
# for some r >= 1
def miillerTest(d, n):
     
    # Pick a random number in [2..n-2]
    # Corner cases make sure that n > 4
    a = 2 + random.randint(1, n - 4);
 
    # Compute a^d % n
    x = power(a, d, n);
 
    if (x == 1 or x == n - 1):
        return True;
 
    # Keep squaring x while one
    # of the following doesn't
    # happen
    # (i) d does not reach n-1
    # (ii) (x^2) % n is not 1
    # (iii) (x^2) % n is not n-1
    while (d != n - 1):
        x = (x * x) % n;
        d *= 2;
 
        if (x == 1):
            return False;
        if (x == n - 1):
            return True;
 
    # Return composite
    return False;
 
# It returns false if n is
# composite and returns true if n
# is probably prime. k is an
# input parameter that determines
# accuracy level. Higher value of
# k indicates more accuracy.
def isPrime( n, k):
     
    # Corner cases
    if (n <= 1 or n == 4):
        return False;
    if (n <= 3):
        return True;
 
    # Find r such that n =
    # 2^d * r + 1 for some r >= 1
    d = n - 1;
    while (d % 2 == 0):
        d //= 2;
 
    # Iterate given nber of 'k' times
    for i in range(k):
        if (miillerTest(d, n) == False):
            return False;
 
    return True;
 
# Driver Code
# Number of iterations
k = 4;
 
print("All primes smaller than 100: ");
for n in range(1,100):
    if (isPrime(n, k)):
        print(n , end=" ");
 
# This code is contributed by mits

All primes smaller than 100: 
2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97 

<h3>VI. KEY EXCHANGE</h3>

Diffie–Hellman key exchange is a method of securely exchanging cryptographic keys over a public channel and was one of the first public-key protocols as conceived by Ralph Merkle and named after Whitfield Diffie and Martin Hellman. DH is one of the earliest practical examples of public key exchange implemented within the field of cryptography. Published in 1976 by Diffie and Hellman, this is the earliest publicly known work that proposed the idea of a private key and a corresponding public key.

In [16]:
def deff():
    # Variables Used
    sharedPrime = 23    # p
    sharedBase = 5      # g

    aliceSecret = 6     # a
    bobSecret = 15      # b

    # Begin
    print( "Publicly Shared Variables:")
    print( "Publicly Shared Prime:" , sharedPrime )
    print( "Publicly Shared Base:" , sharedBase )

    # Alice Sends Bob A = g^a mod p
    A = (sharedBase**aliceSecret) % sharedPrime
    print( "\nAlice Sends Over Public Chanel:" , A )

    # Bob Sends Alice B = g^b mod p
    B = (sharedBase ** bobSecret) % sharedPrime
    print("Bob Sends Over Public Chanel: ", B )

    print( "\n------------\n" )
    print( "Privately Calculated Shared Secret:" )
    # Alice Computes Shared Secret: s = B^a mod p
    aliceSharedSecret = (B ** aliceSecret) % sharedPrime
    print( "Alice Shared Secret: ", aliceSharedSecret )

    # Bob Computes Shared Secret: s = A^b mod p
    bobSharedSecret = (A**bobSecret) % sharedPrime
    print( "Bob Shared Secret:", bobSharedSecret )

<h3>VII. Asymmetric encryption algorithms</h3>

Asymmetric encryption involves the use of multiple keys for data encryption and decryption. To be exact, the asymmetric encryption method comprises two encryption keys that are mathematically related to each other. These keys are known as the public key and private key. As a result, the asymmetric encryption method is also known as “public key cryptography.”

<h6>1. RSA</h6>

In a public-key cryptosystem, the encryption key is public and distinct from the decryption key, which is kept secret (private). An RSA user creates and publishes a public key based on two large prime numbers, along with an auxiliary value. The prime numbers are kept secret. Messages can be encrypted by anyone, via the public key, but can only be decoded by someone who knows the prime numbers

The security of RSA relies on the practical difficulty of factoring the product of two large prime numbers, the "factoring problem". Breaking RSA encryption is known as the RSA problem. Whether it is as difficult as the factoring problem is an open question. There are no published methods to defeat the system if a large enough key is used.

In [24]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
import binascii
keyPair = RSA.generate(3072)
pubKey = keyPair.publickey()

def key_function(p1, p2):
    n={hex(pubKey.n)}
    c={hex(pubKey.e)}
    pubKeyPEM = pubKey.exportKey()

    d={hex(keyPair.d)}
    privKeyPEM = keyPair.exportKey()

    return n, c, d

def encrypt():
    encryptor = PKCS1_OAEP.new(pubKey)
    encrypted = encryptor.encrypt(t)
    me = t^pow(e)
    decrypted_message = me%n

def decrypt():
    decryptor = PKCS1_OAEP.new(keyPair)
    decrypted = decryptor.decrypt(x)

<h6>2. EL GAMAL</h6>

The ElGamal algorithm is an asymmetric cryptographic algorithm based on discrete logarithms. It was created by Taher Elgamal. 
This algorithm is used by the free software GNU Privacy Guard, recent versions of PGP, and other encryption systems, and has never been under patent protection unlike RSA. 
It can be used for encryption and electronic signature. The NIST DSA algorithm is based on ElGamal.

In [25]:
def generate_keys(p,g,a):
    A = g**a%p
    public = (p,g,A)
    private = a
    return (public, private)


def encrypt(message, public, b):
    (p,g,A) = public
    B = g**b%p
    c = message*A**b%p
    return (B, c)

def decrypt(cryptogram, public, private):
    (p,g,A), a = public, private
    (B,c) = cryptogram
    return B**(p-1-a)*c%p

def checkPrime(n):
    m = int(n/2)
    j = 2
    for i in range(0,m):
        if(n % j == 0):
            return 0 # Not Prime
        j +=1
    return 1; # Prime

def powMod(a,b,n):
    x = 1
    y = a
    while b > 0:
        if (int(b % 2) == 1):
            x = int((x * y) % n) # multiplying with base
            #print(x)
        y = int((y * y) % n) # Squaring the base
        #print(y)
        b = int(b/2)
        #print(b)
    return int(x % n)